In [ ]:
# import necessary libraries
import dspy
import mlflow
import os
from dotenv import load_dotenv
import json
from typing import List

print("✅ All imports successful!")

In [ ]:
# configure environment

# Load environment variables
load_dotenv()

# Set up MLflow experiment
mlflow.set_experiment("DSPy-Optimizer-Gemini")

# Enable comprehensive tracing for optimization
mlflow.dspy.autolog(
    log_evals=True,           # Log evaluation results
    log_compiles=True,        # Log optimization process
    log_traces_from_compile=True  # Trace during optimization
)

print("🔍 MLflow tracing enabled with optimization tracking!")
print(f"📊 Experiment: DSPy-Optimizer-Gemini")

In [ ]:
import dspy

# Test Groq connection
lm = dspy.LM(
    'groq/llama-3.3-70b-versatile',
    api_key="your-groq-api-key"
)

print("✅ Groq support loaded via LiteLLM")


In [ ]:

# LIGHTER OPTION: Llama 3.1 8B (fastest, highest limits)
lm = dspy.LM(
    'groq/llama-3.1-8b-instant',  # Fastest with highest RPM/TPM
    api_key=os.getenv('GROQ_API_KEY'),
    max_tokens=1024,
    temperature=0.7
)

print("🚀 Groq (Llama 3.1 8B Instant) configured!")
print(f"📊 Free tier limits:")
print(f"   - Requests: 30/min, 14.4K/day (highest!)")
print(f"   - Tokens: 6K/min, 500K/day")


# Quick test
test = dspy.Predict("question -> answer")
result = test(question="What is the capital of France?")
print(f"\nTest: Capital of France = {result.answer}")
print("✅ Groq connection verified!")

In [ ]:
# Optional: Monitor usage to avoid rate limits
import time

def safe_sleep():
    """Sleep between optimizer trials to respect rate limits"""
    time.sleep(2)  # 2 seconds = max 30 requests/min

# You can add this to the optimizer config in Cell 9 if needed


In [ ]:
# Mock Wikipedia database (since public ColBERT servers are often down)
WIKIPEDIA_MOCK = {
    "python programming": [
        "Python is a high-level programming language created by Guido van Rossum in 1991.",
        "Python emphasizes code readability with significant whitespace and supports multiple programming paradigms.",
        "Python is widely used in web development, data science, and machine learning."
    ],
    "machine learning": [
        "Machine learning is a subset of artificial intelligence that enables systems to learn from data.",
        "Common machine learning algorithms include decision trees, neural networks, and support vector machines.",
        "Machine learning is used in recommendation systems, image recognition, and natural language processing."
    ],
    "eiffel tower": [
        "The Eiffel Tower is located in Paris, France and was completed in 1889.",
        "The tower stands 330 meters tall and was designed by Gustave Eiffel.",
        "It was initially criticized but is now one of the most visited monuments in the world."
    ],
    "albert einstein": [
        "Albert Einstein was a theoretical physicist born in 1879 in Germany.",
        "He developed the theory of relativity, with E=mc² being his most famous equation.",
        "Einstein received the Nobel Prize in Physics in 1921 for his explanation of the photoelectric effect."
    ]
}

def search_wikipedia(query: str) -> List[str]:
    """
    Search Wikipedia for relevant passages.
    Returns top 3 most relevant text passages.
    
    Args:
        query: Search query string
    
    Returns:
        List of relevant text passages from Wikipedia
    """
    query_lower = query.lower()
    
    # Simple keyword matching
    results = []
    for topic, passages in WIKIPEDIA_MOCK.items():
        if any(word in query_lower for word in topic.split()):
            results.extend(passages)
    
    # If no matches, return general info
    if not results:
        results = ["No specific information found. Please try a more specific query."]
    
    return results[:3]  # Return top 3 results


print("📚 Mock Wikipedia search tool created")
print("\n🔍 Test search:")
test_results = search_wikipedia("What is Python?")
for i, result in enumerate(test_results, 1):
    print(f"  {i}. {result[:80]}...")


In [ ]:
# Create ReAct agent with Wikipedia search tool
react_agent = dspy.ReAct(
    signature="question -> answer",
    tools=[search_wikipedia],
    max_iters=5
)

print("🤖 ReAct RAG Agent created")
print("   - Signature: question -> answer")
print("   - Tools: search_wikipedia")
print("   - Max iterations: 5")


In [ ]:
# Create training dataset (small dataset is sufficient for DSPy)
trainset = [
    dspy.Example(
        question="What programming language was created by Guido van Rossum?",
        answer="Python"
    ).with_inputs("question"),
    
    dspy.Example(
        question="When was Python first released?",
        answer="1991"
    ).with_inputs("question"),
    
    dspy.Example(
        question="What is a subset of AI that learns from data?",
        answer="Machine learning"
    ).with_inputs("question"),
    
    dspy.Example(
        question="Where is the Eiffel Tower located?",
        answer="Paris"
    ).with_inputs("question"),
    
    dspy.Example(
        question="Who designed the Eiffel Tower?",
        answer="Gustave Eiffel"
    ).with_inputs("question"),
    
    dspy.Example(
        question="What year was Albert Einstein born?",
        answer="1879"
    ).with_inputs("question"),
    
    dspy.Example(
        question="What is Einstein's famous equation?",
        answer="E=mc²"
    ).with_inputs("question"),
]

# Create validation dataset
valset = [
    dspy.Example(
        question="What does Python emphasize in code?",
        answer="readability"
    ).with_inputs("question"),
    
    dspy.Example(
        question="When was the Eiffel Tower completed?",
        answer="1889"
    ).with_inputs("question"),
    
    dspy.Example(
        question="What prize did Einstein win in 1921?",
        answer="Nobel Prize in Physics"
    ).with_inputs("question"),
    
    dspy.Example(
        question="What are common machine learning algorithms?",
        answer="decision trees"
    ).with_inputs("question"),
]

print(f"📊 Datasets created:")
print(f"   - Training set: {len(trainset)} examples")
print(f"   - Validation set: {len(valset)} examples")
print(f"\n📝 Sample training example:")
print(f"   Question: {trainset[0].question}")
print(f"   Answer: {trainset[0].answer}")


In [ ]:
print("🧪 Testing baseline agent (before optimization)\n")

# Test on a few examples
test_questions = [
    "What programming language was created by Guido van Rossum?",
    "Where is the Eiffel Tower located?",
    "What is Einstein's famous equation?"
]

for question in test_questions:
    result = react_agent(question=question)
    print(f"Q: {question}")
    print(f"A: {result.answer}")
    print("-" * 60)
